# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.

Activate it.

You will find more info about virtual environments in the course content and on the web.

## API Scraping

### A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword.

NOTE: external libraries need to be installed first. Check their documentation.

Check the [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the Requests documentation to:
1. use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is the signification of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [552]:
# import the requests library (1 line)
import requests

# assign the root url (without /status) to the root_url variable for ease of reference (1 line)
root_url = 'https://country-leaders.onrender.com'

# assign the /status endpoint to another variable called status_url (1 line)
status_url = '/status/'

# query the /status endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(f'{root_url}{status_url}')
# check the status_code using a condition and print appropriate messages (4 lines)
if req.status_code == 200:
    print('The client has requested documents from the server. The server has replied to the client and given the client the documents. All is well')

else:
    print('request not succesfull')



The client has requested documents from the server. The server has replied to the client and given the client the documents. All is well


### Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. Check what to do in the Requests [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/).


In [553]:
# Set the countries_url variable (1 line)

countries_url =  'https://country-leaders.onrender.com/countries'

# query the /countries endpoint using the get() method and store it in the req variable (1 line)

req = requests.get(countries_url)

# Get the JSON content and store it in the countries variable (1 line)

countries = req.json()

# display the request's status code and the countries variable (1 line)

print(req,countries)

<Response [403]> {'message': 'The cookie is missing'}


### Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [554]:
# Set the cookie_url variable (1 line)

cookie_url = 'https://country-leaders.onrender.com/cookie/'

# Query the enpoint, set the cookies variable and display it (2 lines)

cookies = requests.get(cookie_url)
print(cookies.text)

{"message":"The cookie has been created"}


Try to query the countries endpoint using the cookie, save the output and print it.

In [555]:
# query the /countries endpoint, assign the output to the countries variable (1 line)

countries = requests.get(countries_url,cookies = {'user_cookie': cookies.cookies['user_cookie']}).json()
# display the countries variable (1 line)

print(countries)


['be', 'fr', 'ma', 'ru', 'us']


Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### Getting the actual data from the API

Query the `/leaders` endpoint.

In [556]:
# Set the leaders_url variable (1 line)

leaders_url = 'https://country-leaders.onrender.com/leaders'

# query the /leaders endpoint, assign the output to the leaders variable (1 line)

leaders = requests.get(leaders_url, cookies = {'user_cookie': cookies.cookies['user_cookie']})

# display the leaders variable (1 line)
print(leaders.json())

{'message': 'Please specify a country'}


It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [557]:
# query the /leaders endpoint using cookies and parameters (take any country in countries)

leaders = requests.get(leaders_url, cookies = {'user_cookie': cookies.cookies['user_cookie']}, params= {'country' : 'ru'})

# assign the output to the leaders variable (1 line)

# display the leaders variable (1 line)

print(leaders.json())


[{'id': 'Q7747', 'first_name': 'Vladimir', 'last_name': 'Putin', 'birth_date': '1952-10-07', 'death_date': None, 'place_of_birth': 'Saint Petersburg', 'wikipedia_url': 'https://ru.wikipedia.org/wiki/%D0%9F%D1%83%D1%82%D0%B8%D0%BD,_%D0%92%D0%BB%D0%B0%D0%B4%D0%B8%D0%BC%D0%B8%D1%80_%D0%92%D0%BB%D0%B0%D0%B4%D0%B8%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B8%D1%87', 'start_mandate': '2000-05-07', 'end_mandate': '2008-05-07'}, {'id': 'Q23530', 'first_name': 'Dmitry', 'last_name': 'Medvedev', 'birth_date': '1965-09-14', 'death_date': None, 'place_of_birth': 'Saint Petersburg', 'wikipedia_url': 'https://ru.wikipedia.org/wiki/%D0%9C%D0%B5%D0%B4%D0%B2%D0%B5%D0%B4%D0%B5%D0%B2,_%D0%94%D0%BC%D0%B8%D1%82%D1%80%D0%B8%D0%B9_%D0%90%D0%BD%D0%B0%D1%82%D0%BE%D0%BB%D1%8C%D0%B5%D0%B2%D0%B8%D1%87', 'start_mandate': '2008-05-07', 'end_mandate': '2012-05-07'}, {'id': 'Q34453', 'first_name': 'Boris', 'last_name': 'Yeltsin', 'birth_date': '1931-02-01', 'death_date': '2007-04-23', 'place_of_birth': 'Butka', 'wikipedia_url

### A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [558]:
leaders_per_country={}

for leader in countries:
    params={}
    params['country'] = leader
    leaders_per_country[leader] = requests.get(leaders_url, cookies = {'user_cookie': cookies.cookies['user_cookie']}, params= params).json()

print(leaders_per_country)    

{'be': [{'id': 'Q12978', 'first_name': 'Guy', 'last_name': 'Verhofstadt', 'birth_date': '1953-04-11', 'death_date': None, 'place_of_birth': 'Dendermonde', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt', 'start_mandate': '1999-07-12', 'end_mandate': '2008-03-20'}, {'id': 'Q12981', 'first_name': 'Yves', 'last_name': 'Leterme', 'birth_date': '1960-10-06', 'death_date': None, 'place_of_birth': 'Wervik', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme', 'start_mandate': '2009-11-25', 'end_mandate': '2011-12-06'}, {'id': 'Q12983', 'first_name': 'Herman', 'last_name': 'None', 'birth_date': '1947-10-31', 'death_date': None, 'place_of_birth': 'Etterbeek', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Herman_Van_Rompuy', 'start_mandate': '2008-12-30', 'end_mandate': '2009-11-25'}, {'id': 'Q14989', 'first_name': 'Léon', 'last_name': 'Delacroix', 'birth_date': '1867-12-27', 'death_date': '1929-10-15', 'place_of_birth': 'Saint-Josse-ten-Noode', 'wikipedia_url': 'htt

In [559]:
# or 1 line
leaders_per_country = {leader: requests.get(leaders_url, cookies={'user_cookie': cookies.cookies['user_cookie']}, params={'country': leader}).json() for leader in countries}


It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [560]:
# < 15 lines
def get_leaders():
    country_leader = {}
    cookies = requests.get(cookie_url)
    countries = requests.get(countries_url,cookies = {'user_cookie': cookies.cookies['user_cookie']}).json()
    for country in countries:
        for key,value in leaders_per_country.items():
            if key == country:
                country_leader[country] = value
                # for mandate in value:
                #     for key,value in mandate.items():
                #         if key == 'last_name':
                #             country_leader[country] = value


                
    return(country_leader)


get_leaders()


{'be': [{'id': 'Q12978',
   'first_name': 'Guy',
   'last_name': 'Verhofstadt',
   'birth_date': '1953-04-11',
   'death_date': None,
   'place_of_birth': 'Dendermonde',
   'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt',
   'start_mandate': '1999-07-12',
   'end_mandate': '2008-03-20'},
  {'id': 'Q12981',
   'first_name': 'Yves',
   'last_name': 'Leterme',
   'birth_date': '1960-10-06',
   'death_date': None,
   'place_of_birth': 'Wervik',
   'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme',
   'start_mandate': '2009-11-25',
   'end_mandate': '2011-12-06'},
  {'id': 'Q12983',
   'first_name': 'Herman',
   'last_name': 'None',
   'birth_date': '1947-10-31',
   'death_date': None,
   'place_of_birth': 'Etterbeek',
   'wikipedia_url': 'https://nl.wikipedia.org/wiki/Herman_Van_Rompuy',
   'start_mandate': '2008-12-30',
   'end_mandate': '2009-11-25'},
  {'id': 'Q14989',
   'first_name': 'Léon',
   'last_name': 'Delacroix',
   'birth_date': '1867-12-27',
   'd

Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [561]:
# 2 lines
leaders_per_country = get_leaders()

print(leaders_per_country)


{'be': [{'id': 'Q12978', 'first_name': 'Guy', 'last_name': 'Verhofstadt', 'birth_date': '1953-04-11', 'death_date': None, 'place_of_birth': 'Dendermonde', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt', 'start_mandate': '1999-07-12', 'end_mandate': '2008-03-20'}, {'id': 'Q12981', 'first_name': 'Yves', 'last_name': 'Leterme', 'birth_date': '1960-10-06', 'death_date': None, 'place_of_birth': 'Wervik', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme', 'start_mandate': '2009-11-25', 'end_mandate': '2011-12-06'}, {'id': 'Q12983', 'first_name': 'Herman', 'last_name': 'None', 'birth_date': '1947-10-31', 'death_date': None, 'place_of_birth': 'Etterbeek', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Herman_Van_Rompuy', 'start_mandate': '2008-12-30', 'end_mandate': '2009-11-25'}, {'id': 'Q14989', 'first_name': 'Léon', 'last_name': 'Delacroix', 'birth_date': '1867-12-27', 'death_date': '1929-10-15', 'place_of_birth': 'Saint-Josse-ten-Noode', 'wikipedia_url': 'htt

## Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [562]:
# 3 lines
wiki_text = requests.get(leaders_per_country['be'][0]['wikipedia_url']).text

print(wiki_text)


<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-toc-available" lang="nl" dir="ltr">
<head>
<meta charset="UTF-8">
<title>Guy Verhofstadt - Wikipedia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-f

Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [563]:
# 3 lines
from bs4 import BeautifulSoup
import lxml

soup = BeautifulSoup(wiki_text, 'lxml')

soup.prettify()


'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-toc-available" dir="ltr" lang="nl">\n <head>\n  <meta charset="utf-8"/>\n  <title>\n   Guy Verhofstadt - Wikipedia\n  </title>\n  <script>\n   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [564]:
# 2 lines
paragraphs = soup.find_all('p')

print(paragraphs)

[<p><b>Guy Maurice Marie-Louise Verhofstadt</b><sup class="reference" id="cite_ref-5"><a href="#cite_note-5"><span class="cite-bracket">[</span>5<span class="cite-bracket">]</span></a></sup> (<span class="ext-phonos"><span class="ext-phonos-PhonosButton noexcerpt oo-ui-widget oo-ui-widget-enabled oo-ui-buttonElement oo-ui-buttonElement-frameless oo-ui-iconElement oo-ui-labelElement oo-ui-buttonWidget" data-nosnippet="" data-ooui='{"_":"mw.Phonos.PhonosButton","href":"\/\/upload.wikimedia.org\/wikipedia\/commons\/transcoded\/d\/dc\/Nl-be_guy_verhofstadt.ogg\/Nl-be_guy_verhofstadt.ogg.mp3","rel":["nofollow"],"framed":false,"icon":"volumeUp","label":{"html":"uitspraak"},"data":{"ipa":"","text":"","lang":"nl","wikibase":"","file":"Nl-be guy verhofstadt.ogg"},"classes":["ext-phonos-PhonosButton","noexcerpt"]}' id="ooui-php-1"><a aria-label="Geluid afspelen" class="oo-ui-buttonElement-button" href="//upload.wikimedia.org/wikipedia/commons/transcoded/d/dc/Nl-be_guy_verhofstadt.ogg/Nl-be_guy_v

If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one
  * store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
  * exit the loop

In [565]:
# <10 lines
import re

first_paragraph =[]
for tag in soup.find_all('p'):
    if tag.b != None:
        first_paragraph.append(tag.text)

print(first_paragraph)


['Guy Maurice Marie-Louise Verhofstadt[5] (uitspraakⓘ) (Dendermonde, 11 april 1953) is een Belgisch politicus voor de Open Vlaamse Liberalen en Democraten (Open Vld). Hij was premier van België van 12 juli 1999 tot 20 maart 2008 in drie regeringen. Hij beëindigde zijn actieve politieke carrière in het Europees Parlement, waar hij van 2009 tot 2019 fractieleider van de Alliantie van Liberalen en Democraten voor Europa (ALDE) was.[6]\n']


At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [566]:
# 10 lines
def get_first_paragraph(wikipedia_url):
    print(wikipedia_url)
    soup = BeautifulSoup(wikipedia_url.text, 'lxml')
    first_paragraph =[]
    for tag in soup.find_all('p'):
        if tag.b != None:
            tag = tag.text
            first_paragraph.append(tag)
    return first_paragraph
    


In [567]:
# Test: 3 lines
get_first_paragraph(requests.get(leaders_per_country['be'][1]['wikipedia_url']))

<Response [200]>


['Yves Camille Désiré Leterme (Wervik, 6 oktober 1960) is een Belgisch politicus. Hij was onder meer Vlaams minister-president en premier van België. Hij was lang boegbeeld van de CD&V en boegbeeld van het toenmalige kartel CD&V/N-VA.\n']

### Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [568]:
# 3 lines

import re

# pristine_tag = re.sub(r'\[.*?\]|<.*?>', '', tag)
# pristine_tag = re.sub(r'\xa0', ' ',pristine_tag)

Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [569]:
# 10 lines
def get_first_paragraph(wikipedia_url):
    print(wikipedia_url)
    soup = BeautifulSoup(wikipedia_url.text, 'lxml')
    first_paragraph =[]
    for tag in soup.find_all('p'):
        if tag.b != None:
            tag = tag.text
            pristine_tag = re.sub(r'\[.*?\]|<.*?>|\(.*ⓘ\)', '', tag)
            pristine_tag = re.sub(r'\xa0', ' ',pristine_tag)
            first_paragraph.append(pristine_tag)
    return first_paragraph


get_first_paragraph(requests.get(leaders_per_country['ru'][2]['wikipedia_url']))


<Response [200]>


['Бори́с Никола́евич Е́льцин (1 февраля 1931 (1931-02-01), Бутка, Буткинский район, Уральская область, РСФСР, СССР — 23 апреля 2007, Москва, Россия) — советский и российский партийный, государственный и политический деятель, первый всенародно избранный Президент Российской Федерации (1991—1999); в ноябре 1991 — июне 1992 года одновременно возглавлял правительство. С марта по май 1992 года исполнял обязанности министра обороны Российской Федерации.\n',
 'См. также: Список наград и почётных званий Бориса Ельцина\n']

Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [570]:
# < 20 lines


## Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [571]:
cookie_url = 'https://country-leaders.onrender.com/cookie/'
leaders_url = 'https://country-leaders.onrender.com/leaders'
countries_url =  'https://country-leaders.onrender.com/countries'

# query the /leaders endpoint, assign the output to the leaders variable (1 line)

cookies = requests.get(cookie_url)
countries = requests.get(countries_url,cookies = {'user_cookie': cookies.cookies['user_cookie']}).json()

In [572]:
# < 20 lines
def get_leaders():

    while True:
    
        try:
            cookies = requests.get(cookie_url)
            countries = requests.get(countries_url,cookies = {'user_cookie': cookies.cookies['user_cookie']}).json()
            break
        except requests.exceptions.RequestException as e:
            print ('There was an error:', e)
            print(cookies.status_code)

    leaders_per_country = {leader: requests.get(leaders_url, cookies={'user_cookie': cookies.cookies['user_cookie']}, params={'country': leader}).json() for leader in countries}
    country_leader = {}
    for country in countries:
        for key,value in leaders_per_country.items():
            if key == country:
                country_leader[country] = value
    
    for country, leaders in country_leader.items():
        for leader in leaders:
            leader['first_paragraph'] = get_first_paragraph(requests.get(leader['wikipedia_url']))
           
    print (country_leader)
    


In [573]:
get_leaders()


<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [574]:
# < 25 lines



Check the output of your function again.

In [575]:
# Check the output of your function (1 line)


Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [576]:
# < 20 lines


Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [577]:
# <25 lines


Test your new functions.



## Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [578]:
# 3 lines


Make sure the file can be read back. Write the code to read the file. And check the variables are the same.

In [579]:
# 3 lines


Make a function `save(leaders_per_country)` to call this code easily.

In [580]:
# 3 lines


In [581]:
# Call the function (1 line)


## Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## To go further

If you want to practice scraping, you can read this section and tackle the exercise.

You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!